In [1]:
import sys
sys.path.append('../')
import env

In [2]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression as LR

In [3]:
READ_FILE = 'next_move_area'

In [4]:
target_csv_list = ['od', 'estimate']

In [5]:
# エリア番号を線形的な数から、iとjで回した数のようにする
def convert_area_to_contour(base, read):
    for r in np.asanyarray(read):
        area_id = int(r[0])
        row = str(area_id // env.AREA_ONE_LENGTH())
        col = str(area_id % env.AREA_ONE_LENGTH()) + '0'
        base.loc[row, col] = r[1]
        
    return base

In [6]:
base = create_base_dataframe()

def main(args, array):
    for _area in env.get_area_list():
        df = pd.read_csv(env.get_full_path(READ_FILE, args, any=_area),
                         encoding='Shift_JISx0213')
        df = convert_area_to_contour(base.copy(), df.copy())
    #                 df = df / df.sum().sum()
        df = df.sort_index(ascending=False)
        df.fillna(0, inplace=True)

        _area = env.area_to_contour(_area)
        array[args.dir][args.ratio][args.seed][args.csv][_area]= df
        
csv_array = {}
env.for_default_init(main, csv_array, target_csv_list)

NameError: name 'create_base_dataframe' is not defined

In [ ]:
people = 'p10000'
ratio = 'r4'
seed = 's123'

In [ ]:
def create2d(args, array):
    for _area in env.get_area_list():
        df = pd.read_csv(env.get_full_path(READ_FILE, args, any=_area),
                         encoding='Shift_JISx0213')
        _area = env.area_to_contour(_area)
        array[args.dir][args.ratio][args.seed][args.csv][_area] = df
        
csv_array_2d = {}
env.for_default_init(create2d, csv_array_2d, target_csv_list)

In [ ]:
from sklearn.linear_model import LinearRegression as LR
def show_score(x, y):
    x = x.values.reshape(-1, 1)
    y = y.values.reshape(-1, 1)
    model = LR()
    model.fit(x, y)
    score = model.score(x, y)
    print(score)
    
    return x, y

In [ ]:
from sklearn.linear_model import LinearRegression as LR
from sklearn.preprocessing import PolynomialFeatures

score_array = []
def create_score2d(x, y):
    x = x.values.reshape(-1, 1)
    y = y.values.reshape(-1, 1)
    
    quadratic = PolynomialFeatures(degree = 2)
    X_quad = quadratic.fit_transform(x)
    X_fit = np.arange(trainX.min(), trainX.max(), 1)[:, np.newaxis]
    model = LR()

    # 2次の項を追加、予測値、R^2を評価
    mod_quad = model.fit(X_quad, y)
    y_quad_fit = mod_quad.predict(quadratic.fit_transform(X_fit))
    r2_quad = model.score(X_quad, y)

#     plt.figure(figsize=(9,6), dpi=600)
    plt.scatter(x, y)
    plt.plot(X_fit, y_quad_fit, 'orange')
    print(r2_quad)
    score_array.append(r2_quad)
#     plt.savefig('hoge.png')

In [ ]:
# plt.figure(figsize=(9,6), dpi=600)
mean_array = []
score_array = []

from sklearn.metrics import r2_score
import warnings
warnings.filterwarnings('ignore')

for _area in env.get_area_list():
    _area = env.area_to_contour(_area)
    target = csv_array_2d[people][ratio][seed]['estimate'][_area].copy()
    od = csv_array_2d[people][ratio][seed]['od'][_area].copy()

    trainX = target['people']
    trainY = od['people']
#     plt.figure(figsize=(9,6), dpi=600)

    corr = np.corrcoef(trainX, trainY)[0,1]
#     r2 = r2_score(trainX, trainY)
    if ((np.isnan(corr) == False) and trainX.mean().mean() > 4):

        plt.plot(trainX, trainY, 'o')
        order=2
        sns.regplot(x=trainX, y=trainY, order=order, ci=0)
        z = np.polyfit(trainX, trainY, order)
        z = np.squeeze(z)
        p = np.poly1d(z)
    #         print('y = ', p)
    
        trainX = trainX.values.reshape(-1, 1)
        trainY = trainY.values.reshape(-1, 1)
        model = LR()
        model.fit(trainX, trainY)
        score = model.score(trainX, trainY)
        score_array.append(score)

        if score >= 0 :
            mean = trainX.mean().mean()
            mean_array.append(mean)
            print(_area, '相関: ', round(corr, 3),  '| R^2 ', round(score, 3), '| 平均: ', round(mean, 3))
            
    #         plt.savefig('{0}_mobile_include_id.png'.format(_area))

mean = 0
for i in mean_array:
    mean += i
print('mean平均: ', mean / len(mean_array))

score = 0
for i in score_array:
     score += i
print('score平均: ', score / len(score_array))

In [ ]:
_area = '440'
target = csv_array_2d[people][ratio][seed]['estimate'][_area].copy()
od = csv_array_2d[people][ratio][seed]['od'][_area].copy()

trainX = target['people']
trainY = od['people']
#     plt.figure(figsize=(9,6), dpi=600)
mean_array = []

corr = np.corrcoef(trainX, trainY)[0,1]
if (np.isnan(corr) == False):

    plt.plot(trainX, trainY, 'o')
    order=2
    sns.regplot(x=trainX, y=trainY, order=order, ci=0)
    z = np.polyfit(trainX, trainY, order)
    z = np.squeeze(z)
    p = np.poly1d(z)
#         print('y = ', p)

    trainX = trainX.values.reshape(-1, 1)
    trainY = trainY.values.reshape(-1, 1)
    model = LR()
    model.fit(trainX, trainY)
    score = model.score(trainX, trainY)

    if score >= 0 :
        mean = trainX.mean().mean()
        mean_array.append(mean)
        print(_area, '相関: ', round(corr, 3),  '| R^2 ', round(score, 3), '| 平均: ', round(mean, 3))